TODO list:
- scale up the exports so I can do this everywhere
- Examine where the most changes have happened, visualize this on a map
- Expand more on this .ipynb about each of the cells, processing steps, etc

Importing useful packages

In [22]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import ipyleaflet
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display

ee.Initialize()

Functions for handling geometry and data

In [23]:
def pd_shp_to_ee_poly(shp):
    """Converts Polygon from GeoPandas to a ee.Geometry.Polygon
    object suitable for use within Google Earth Engine."""
    xs, ys = shp.exterior.coords.xy
    shp_list = [[x, y] for x, y in zip(xs, ys)]
    roi = ee.Geometry.Polygon(shp_list, None, False)
    return roi


def mask_classes(image, vals_to_keep):
    """Masks values of the image to only include those
    within vals_to_keep."""
    masks = []
    finalMask = ee.Image(0)

    for val in vals_to_keep:
        masks.append(image.eq(val))
    
    for mask in masks:
        finalMask = finalMask.Or(mask) 
    
    return image.updateMask(finalMask)


def get_county_roi(county_name):
    """Returns a ee.Geometry.Polygon object representing
    a particular county within Georgia, along with the
    centroid of that object."""
    ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
    county_shp = ga_counties[ga_counties["NAME10"] == county_name].geometry.values[0]
    xs, ys = county_shp.centroid.coords.xy
    county_roi = pd_shp_to_ee_poly(county_shp).simplify(maxError = 1)

    return county_roi, (xs[0], ys[0])


def get_labels(collection, class_name):
    """Returns a data frame containing the band values/class/corresponding
    palette color."""
    class_vals = collection.first().get(f"{class_name}_class_values").getInfo()
    class_labels = collection.first().get(f"{class_name}_class_names").getInfo()
    class_palette = collection.first().get(f"{class_name}_class_palette").getInfo()

    class_desc = [lab.split(':')[1] for lab in class_labels]
    class_labels = [lab.split(':')[0] for lab in class_labels]

    class_df = pd.DataFrame({
        'layer_vals': class_vals,
        'labels': class_labels,
        'class_description': class_desc,
        'palette': class_palette
    })

    return class_df


def get_legend_keys_values(class_df, vals_to_keep):
    """Returns a reduced legend from class_df according to the
    values in the list vals_to_keep."""
    legend_keys = list(class_df[class_df.layer_vals.isin(vals_to_keep)].labels)
    legend_keys = [leg.split(':')[0] for leg in legend_keys]
    legend_colors = list(class_df[class_df.layer_vals.isin(vals_to_keep)].palette)
    return legend_keys, legend_colors


def union_polygons(poly_ls):
    """Given a list of polygons, returns the union of all of them."""
    basePoly = poly_ls[0]
    for poly in poly_ls[1:]:
        basePoly = basePoly.union(poly, maxError = 1)

    return basePoly


def rect_from_corners(tl, br):
    """Returns a ee.Geometry.Rectangle object from the
    top left and bottom right corners."""
    return ee.Geometry.Polygon(
        [[tl[0], tl[1]],
        [br[0], tl[1]],
        [br[0], br[1]],
        [tl[0],br[1]]], None, False
    )

In [24]:
# List of "main" counties in Atlanta
county_list = [
    'Cherokee', 'Clayton', 'Cobb', 
    'DeKalb', 'Douglas', 'Fayette', 
    'Forsyth', 'Fulton', 'Gwinnett', 
    'Henry','Rockdale'
]

# List of counties in the census specified statistical area
# for Atlanta
larger_county_list = [
    'Fulton', 'Gwinnett',
    'Cobb', 'DeKalb', 'Clayton',
    'Cherokee', 'Forsyth',
    'Henry', 'Paulding', 'Coweta',
    'Douglas', 'Fayette', 'Carroll',
    'Newton', 'Bartow', 'Walton', 
    'Rockdale', 'Barrow', 'Spalding', 'Pickens', 
    'Haralson', 'Dawson', 'Butts', 
    'Meriwether', 'Morgan',
    'Pike', 'Lamar', 'Jasper', 'Heard'
]

# Loading shape files for counties and tracts within Georgia
ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
ga_tracts = gpd.read_file("ga-tracts-2019/tl_2019_13_tract.shp")

In [25]:
at_counties = ga_counties[ga_counties["NAME10"].isin(county_list)]
at_counties.head()

,OBJECTID,STATEFP10,COUNTYFP10,GEOID10,NAME10,NAMELSAD10,totpop10,WFD,RDC_AAA,MNGWPD,...,MSA,F1HR_NA,F8HR_NA,Reg_Comm,Acres,Sq_Miles,Label,GlobalID,last_edite,geometry
5,6,13,113,13113,Fayette,Fayette County,106567,Y,Y,Y,...,Y,Y,Y,Atlanta Regional Commission,127543.000000,199.285995,FAYETTE,{0089049C-AF9E-48C9-83D8-75FE86DFE045},None,"POLYGON ((-84.55686 33.52841, -84.55136 33.529..."
11,12,13,211,13211,Morgan,Morgan County,17868,N,N,N,...,Y,N,N,Northeast Georgia,226933.000000,354.583008,MORGAN,{77264900-C504-4F22-BA88-409240758961},None,"POLYGON ((-83.65341 33.50795, -83.66298 33.514..."
19,20,13,045,13045,Carroll,Carroll County,110527,N,N,N,...,Y,N,N,Three Rivers,322447.000000,503.824005,CARROLL,{33DEDEB8-8E54-40A7-A237-847405D84720},2015-10-14,"POLYGON ((-84.97214 33.79979, -84.97211 33.799..."
20,21,13,247,13247,Rockdale,Rockdale County,85215,Y,Y,Y,...,Y,Y,Y,Atlanta Regional Commission,84525.703125,132.070999,ROCKDALE,{1146BC41-D714-4074-BFCC-B1EA52C17B96},None,"POLYGON ((-83.93160 33.65087, -83.93163 33.650..."
23,24,13,067,13067,Cobb,Cobb County,688078,N,Y,Y,...,Y,Y,Y,Atlanta Regional Commission,220455.000000,344.459991,COBB,{670FE76B-423B-4211-BFB0-8A9145225470},None,"POLYGON ((-84.72423 33.90360, -84.72423 33.903..."


In [26]:
# Merging info from the counties into ga_tracts, keeping
# only the tracts that are within the counties of interest
at_tracts = ga_tracts.merge(
    ga_counties[['COUNTYFP10', 'NAME10']],
    left_on = "COUNTYFP",
    right_on = "COUNTYFP10",
).query(
    "NAME10 in @larger_county_list"
).reset_index()

at_tracts.head()

,index,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,COUNTYFP10,NAME10
0,5,13,067,031209,13067031209,312.09,Census Tract 312.09,G5020,S,3407093,4863,+33.8558429,-084.4944273,"POLYGON ((-84.51004 33.85250, -84.50997 33.852...",067,Cobb
1,6,13,067,031207,13067031207,312.07,Census Tract 312.07,G5020,S,2775451,49508,+33.8710765,-084.4701703,"POLYGON ((-84.47968 33.86431, -84.47805 33.868...",067,Cobb
2,7,13,067,030507,13067030507,305.07,Census Tract 305.07,G5020,S,6091182,84766,+34.0070818,-084.5138141,"POLYGON ((-84.52825 34.01994, -84.52816 34.019...",067,Cobb
3,8,13,067,030506,13067030506,305.06,Census Tract 305.06,G5020,S,8002577,41211,+33.9974216,-084.5386849,"POLYGON ((-84.55801 33.98199, -84.55793 33.982...",067,Cobb
4,9,13,067,030259,13067030259,302.59,Census Tract 302.59,G5020,S,3038794,9772,+34.0368452,-084.6190164,"POLYGON ((-84.62792 34.03201, -84.62790 34.032...",067,Cobb


Load geometries and class_df

In [27]:
# Create unified geometry for Georgia
poly_list = []

for county in larger_county_list:
    county_shp = ga_counties[ga_counties["NAME10"] == county].geometry.values[0]
    poly_list.append(pd_shp_to_ee_poly(county_shp).simplify(maxError = 1))

atlanta_roi = union_polygons(poly_list).simplify(maxError = 1)

In [28]:
# Restrict collection to Atlanta, extract information about class labels
# for future reference
collection = (
    ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
    .map(lambda image: image.clip(atlanta_roi))
)

class_df = get_labels(collection, 'landcover')
class_df.to_csv("data/ncld_class_labels.csv")
class_df

,layer_vals,labels,class_description,palette
0,11,Open water,"areas of open water, generally with less than...",466b9f
1,12,Perennial ice/snow,areas characterized by a perennial cover of i...,d1def8
2,21,"Developed, open space",areas with a mixture of some constructed mate...,dec5c5
3,22,"Developed, low intensity",areas with a mixture of constructed materials...,d99282
4,23,"Developed, medium intensity",areas with a mixture of constructed materials...,eb0000
5,24,Developed high intensity,highly developed areas where people reside or...,ab0000
6,31,Barren land (rock/sand/clay),"areas of bedrock, desert pavement, scarps, ta...",b3ac9f
7,41,Deciduous forest,areas dominated by trees generally greater th...,68ab5f
8,42,Evergreen forest,areas dominated by trees generally greater th...,1c5f2c
9,43,Mixed forest,areas dominated by trees generally greater th...,b5c58f


Write function working on different geometry areas and images, record
roads separately rather than simply as a impervious surface.

In [29]:
def remove_roads(image):
    """Removes impervious surfaces corresponding
    to roads from the NCLD collection image."""
    image_imp = image.select('impervious_descriptor')
    image_land = image.select('landcover')

    # Create a mask for impervious surfaces
    impervious_class_roads = [20, 21, 22]

    land_mask = (
        mask_classes(image_imp, impervious_class_roads)
        .mask().Not()
    )

    return image_land.mask(land_mask)

def calculate_histogram_over_region(geometry, image):
    """Given an image, assumed to be from the NLCD collection, and a
    geometry region, return the histogram of pixel values for each class."""
    vals = (
        image.select('landcover').clip(geometry)
        .reduceRegion(
            ee.Reducer.frequencyHistogram()
        )
        .getInfo()
    )

    vals_df = (
        pd.DataFrame.from_dict(vals)
        .rename_axis(index = 'values')
        .reset_index()
    ).astype({'values': 'float64', 'landcover': 'float64'})

    return vals_df

In [35]:
years = [2001, 2004, 2006, 2008, 2011, 2013, 2016, 2019]

at_tracts_fc = geemap.geopandas_to_ee(at_tracts.sample(10))
collection_list = collection.toList(collection.size())

num_rows = at_tracts.shape[0]
df_list = []

for year_idx, year in enumerate(years):
    image = ee.Image(collection_list.get(year_idx))
    image = remove_roads(image)
    
    new_fc = image.reduceRegions(
        collection = at_tracts_fc,
        reducer = ee.Reducer.frequencyHistogram()
    )

    geemap.ee_export_vector_to_drive(
        new_fc,
        description = f'ncld_histogram_{year}',
        folder = 'data',
        file_format = 'csv',
        selectors = ['TRACTCE', 'histogram']
    )


/home/ad/anaconda3/envs/comp-journal/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Exporting ncld_histogram_2001...
Exporting ncld_histogram_2004...
Exporting ncld_histogram_2006...
Exporting ncld_histogram_2008...
Exporting ncld_histogram_2011...
Exporting ncld_histogram_2013...
Exporting ncld_histogram_2016...
Exporting ncld_histogram_2019...
